<div style="float:left;font-size:20px;">
    <h1>PCA</h1>
</div><div style="float:right;"><img src="../assets/banner.jpg"></div>

<hr>

In [9]:
import numpy as np
import sklearn as sk
import altair as alt
alt.data_transformers.disable_max_rows(); # Disable max rows == 5000 limitation
alt.data_transformers.enable('json');     # Save data to JSON, rather than to notebook to keep size small

In [10]:
from sklearn.datasets import make_classification
X, Y = sk.datasets.make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)

In [11]:
X

array([[ 0.55372733, -1.36468108,  2.07106629, ..., -2.06257343,
         0.89864144, -1.04435104],
       [ 0.88808176,  0.64347898, -0.69729104, ..., -2.38303113,
         0.92263525, -0.08859627],
       [-1.13255996,  1.37390753, -1.1035627 , ..., -0.4398791 ,
        -0.28261564,  0.78464676],
       ...,
       [-0.09374991, -0.32647105,  0.15913969, ..., -0.62737913,
         0.85179233, -0.37813943],
       [ 0.26648521,  0.25075628,  2.28648809, ...,  0.88916947,
        -2.21715985,  0.63234516],
       [-0.40472403,  0.86432089,  0.03871788, ...,  0.00708372,
         1.87241111,  0.54421312]])

In [ ]:
from vega_datasets import data

source = data.iris()
brush = alt.selection(type='interval', resolve='global')

base = alt.Chart(source).mark_point().encode(
    y='sepalLength',
    color=alt.condition(brush, 'species', alt.ColorValue('gray')),
).add_selection(
    brush
).properties(
    width=250,
    height=250
)

In [5]:
X.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'